# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.0 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446659 sha256=dca687d90b735b6e6738bd02035fedb39e118b4ff0595e16ffa85ad20762d648
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed

### Importing the libraries

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [ ]:
class Network(nn.Module):
  def __init__(self,state_size,action_size,seed=42):
    super(Network,self).__init__()
    self.seed= torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, 64 )
    self.fc2 = nn.Linear(64 , 64 )
    self.fc3 = nn.Linear(64 , action_size )

  def forward(self,state):
    x = self.fc1(state)
    x = F.relu(x)

    x = self.fc2(x)
    x = F.relu(x)

    return self.fc3(x)


## Part 2 - Training the AI

### Setting up the environment

In [ ]:
import gymnasium as gym
env = gym.make("LunarLander-v2")
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print ("State Shape = ",state_shape)
print ("State Size = ",state_size)
print ("Number of actions = ",number_actions)

State Shape =  (8,)
State Size =  8
Number of actions =  4


### Initializing the hyperparameters

In [ ]:
learning_rate = 5e-4  #best value after many research
minibatch_size = 100 #just a random optimal value
discount_factor = 0.99  #gamma
replay_buffer_size = int(1e5) #100,000 experiences
interpolation_parameter = 1e-3 #researched value



### Implementing Experience Replay

In [ ]:
class ReplayMemory(object):
  def __init__(self, capacity ): #capacity of memory
       self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu" ) #to use gpu if its not collab
       self.capacity = capacity #max size of memory buffer
       self.memory = [] #experiences containing state action reward etc

  #add memories
  def push(self,event): #event = experience itself
       self.memory.append(event) #making sure capacity dont exceed
       if len(self.memory) > self.capacity:
        del self.memory[0]  #delete oldest event

  def sample(self, batch_size):
    experiences = random.sample(self.memory, k= batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device) #loop within a list . stack first element in experiences list (states have index 0)
    # above we converted numpy array to pytorch sensor
    # .to(self.device) added to move states to designated computing device
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device) #action is 0 ,1 2 or 3 so long int
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states ,actions ,rewards , dones






### Implementing the DQN class

In [ ]:
class Agent():
  def __init__(self, state_size,action_size):
     self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu" )
     self.state_size = state_size
     self.action_size = action_size
     # create 2 network local and target
     self.local_qnetwork = Network(state_size,action_size).to(self.device)
     self.target_qnetwork = Network(state_size,action_size).to(self.device)
     self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate) #weight of network
     self.memory = ReplayMemory(replay_buffer_size)
     self.t_step=0

  def step(self, state , action , reward , next_state, done):
       self.memory.push(( state , action , reward , next_state, done)) # extra bracket for tuple
       #learn every 4 steps
       self.t_step =  (self.t_step + 1 )% 4 #to reset it adn increment
       if  self.t_step == 0:
        # first check if mini batch size more than 100
        if len(self.memory.memory) > minibatch_size: # first memory is object of ReplayMemory other one is attribute
          experiences = self.memory.sample(100) #experiences is local
          #learn from experiences
          self.learn(experiences ,discount_factor)

  #act method select an action based on the given state and epsilon
  def act(self, state, epsilon = 0.):
    #we have 8 dimensions in state , we need to add extra dimension which corresponds to batch
    # extra dimension will say which state the action belongs to
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device) #added at beginning
    # forawrd pass state through local q net to get action values , set qnet to evaluation mode
    self.local_qnetwork.eval()
    # make sure we are in inference mode not in training
    # no gradient computations (make sure its disabled)
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    # go back to training mode
    self.local_qnetwork.train()
    # epsilon greedy action selection policy
    # random num > epsilon select highest q value else random action
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size)) #select among 0 1 2 and 3

  def learn(self,experience,discount_factor):
    states,next_states,actions,rewards,dones = experience #unpacking
    next_qtargets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + (discount_factor * next_qtargets * (1-dones))
    q_expected =  self.local_qnetwork(states).gather(1,actions)
    loss = F.mse_loss(q_expected,q_targets)
    self.optimizer.zero_grad() # a reset
    # now back propogate losses
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork,self.target_qnetwork, interpolation_parameter)

  def soft_update(self, local_model , target_model, interpolation_parameter):
    for target_param , local_param in zip(target_model.parameters(),local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter)*target_param.data)




### Initializing the DQN agent

In [ ]:
agent = Agent(state_size,number_actions)

### Training the DQN agent

In [ ]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value = 1.0
epsilon_ending_value = 0.01
epsilon_decay_value = 0.095
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes+1):
  # reset environment at beginning of every episode
  state, _ = env.reset() # _ is to discard some stuff
  score = 0
  for t in range (maximum_number_timesteps_per_episode):
    action = agent.act(state,epsilon)
    next_state , reward, done , _,_ = env.step(action)
    agent.step(state,action,reward,next_state,done) #learn
    state = next_state #update state
    score += reward #since its cumulative
    if done:
      break #to finish episode
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value , epsilon_decay_value * epsilon)

  print("\rEpsiode {}\tAverage Score: {:.2f}".format(episode,np.mean(scores_on_100_episodes)),end = "")
  if episode % 100 == 0:
      print("\rEpsiode {}\tAverage Score: {:.2f}".format(episode,np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
      print("\nEnvironment solved in  {:d} episodes!\tAverage Score: {:.2f}".format(episode - 100,np.mean(scores_on_100_episodes)))
      torch.save(agent.local_qnetwork.state_dict() , "checkpoint.pth") #to save parameters
      # exit training
      break


Epsiode 100	Average Score: -108.83
Epsiode 200	Average Score: -63.24
Epsiode 300	Average Score: -38.32
Epsiode 400	Average Score: -40.20
Epsiode 500	Average Score: -30.83
Epsiode 600	Average Score: -26.37
Epsiode 700	Average Score: -27.76
Epsiode 800	Average Score: -18.05
Epsiode 900	Average Score: -13.97
Epsiode 1000	Average Score: -3.89
Epsiode 1100	Average Score: 26.62
Epsiode 1200	Average Score: 167.15
Epsiode 1300	Average Score: 199.83
Epsiode 1301	Average Score: 200.90
Environment solved in  1201 episodes!	Average Score: 200.90


## Part 3 - Visualizing the results

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
